# Non-negative Matrix Factorization on Radiomics Features

NMF is performed with the [R package "NMF"](https://doi.org/10.1186/1471-2105-11-367) (Gaujoux R, Seoighe C. A flexible R package for nonnegative matrix factorization. BMC Bioinformatics. 2010;11:367)

For *n* measured features, and *p* patients ("samples" in micro-assay terms), we have a feature matrix $X$ of size (*n* x *p*). NMF aims to find a decomposition 

$X \approx WH$,

so that $W$ and $H$ are non-negative matrices with sizes (*n* x *r*) and (*r* x *p*), respectively, where the rank *r* is a positive integer.

$W$, the basis, contains *r* meta-features (in the literature sometimes called meta-genes).

$H$ contains, for each patient, the coefficients of each meta-feature. 





In our case, we start with a table of Radiomics features per patient in the following format (to be read from csv file):

| PatientID | Feature_0 | Feature_1 | Feature_2 | Feature_3 | ... | Feature_n |
| --------- | --------- | --------- | --------- | --------- | --- | --------- |
| Patient_A | x | x | x | x | ... | x |
| Patient_B | x | x | x | x | ... | x |
| Patient_C | x | x | x | x | ... | x |

NB This is transposed from the NMF formulation above, i.e. respresenting $X^T$, with patients as rows and features as columns.

An ExpressionSet object is created as a data container, with feature values as "assayData" (here we transpose to obtain the "patients as columns" format).

To run NMF, call `nmf()` on the ExpressionSet object, with the desired rank k (or range as min_k:max_k) and number of runs to perform. 


We aim to find the best rank *r* that groups patients into consistent clusters by considering the consensus (see [Brunet JP, Tamayo P, Golub TR, Mesirov JP. Metagenes and molecular pattern discovery using matrix factorization. Proc Natl Acad Sci U S A. 2004;101(12):4164-4169.](doi:10.1073/pnas.0308531101) for reference.

In [ ]:
install.packages('NMF')

if (!require("BiocManager"))
+ install.packages("BiocManager")

library("Biobase")
library(NMF)

library(repr)
options(repr.plot.width=20, repr.plot.height=10)

In [ ]:
feature_file <- file.path("/path/to/feature_file.csv")
features <- as.matrix(read.table(feature_file, header=TRUE, sep=",", row.names="PatientID", as.is=TRUE))

In [ ]:
feature_set <- ExpressionSet(assayData=t(features))
feature_set

In [ ]:
# run nmf for ranks 2, 3, 4, 5, 6 and form consensus of 50 runs
res <- nmf(feature_set, 2:6, nrun=50, seed=123456)

#### Plot of Clustering Performance Measures

In [ ]:
plot(res)

In [ ]:
res$'measures'

#### Plot Consensus Maps

In [ ]:
consensusmap(res, annCol=feature_set, labCol=NA, labRow=NA)

#### Results for Individual Ranks
Access via 'fit' attribute or rerun `nmf()` with single value for k.

In [ ]:
# e.g. for rank = 4:
res_4 <- res $'fit'$'4'

##### Coefficients
For each patient, get $H$, the contribution of meta-features with `coef()` or `scoef()` (normalised to sum to 1).

In [ ]:
coef(res_4)

In [ ]:
scoef(res_4)

##### Basis
For each meta-feature, get $W$, the basis: 

In [ ]:
basis(res_4)

In [ ]:
basismap(res_4)